In [1]:
%reset -f

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
import time
import tools

In [2]:
# Parameters to tweak
lr = 1e-3
batchSize = 64
epochs = 2
loss_fn = nn.CrossEntropyLoss()

In [3]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

# Create 64 batches
train_dataloader = DataLoader(training_data, batch_size=batchSize)
test_dataloader = DataLoader(test_data, batch_size=batchSize)
len(test_dataloader)

100%|██████████| 9912422/9912422 [00:00<00:00, 21139221.19it/s]


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 15159713.01it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw



100%|██████████| 4542/4542 [00:00<?, ?it/s]


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



157

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.Sigmoid(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
def train_data(model, optimizer):
    for xb, yb in train_dataloader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss = loss.item()
    print(f"Train loss: {loss:>7f}")

In [7]:
def test_data(model):
    num_batches = len(test_dataloader)
    size = len(test_dataloader.dataset)
    test_loss, corrects = 0, 0

    with torch.no_grad():
        for xb, yb in test_dataloader:
            preds = model(xb)
            test_loss += loss_fn(preds, yb).item()
            corrects += (preds.argmax(1) == yb).type(torch.float).sum().item()

    test_loss /= num_batches
    # test_loss = lo
    corrects /= size
    print(f"Test loss: \n Accuracy: {(100*corrects):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return corrects

In [8]:
model = NeuralNetwork().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
epoch_pro = []
epoch_performance = []
for t in range(epochs):
    start_time = time.time()

    train_data(model, optimizer)
    epoch_pro.append(test_data(model))
    epoch_performance.append(time.time()-start_time)
        

Train loss: 2.277262
Test loss: 
 Accuracy: 29.9%, Avg loss: 2.265239 

Train loss: 2.245332
Test loss: 
 Accuracy: 43.8%, Avg loss: 2.228994 



In [9]:
tools.saveResult(epochs,lr,"crossentropyloss",batchSize,epoch_pro,epoch_performance, "model_cross_entropy_sigmoid")